
# "Battle of the Neighbourhoods" : London Pleasantness Index

## by kingnif

#### Introduction

When looking for potential areas to live in London, different people have different priorities. It's relatively easy to find out which areas are generally good for commuting, or for schools, but what about neighbourhoods that people are happy to live in, that have lots of park space, that are good for socialising? What about neighbourhoods that are "pleasant"? For this research, I have decided to create a _Pleasantness Index_ based on three factors:

- Happiness of local people
- Amount of green space
- Number of coffee shops 

I have chosen not to investigate some factors that are obvious (such as average house prices), and instead have decided to investigate something I have not seen before - a [very simple] pleasantness score! I hope that, hypothetically, this would appeal to people of a similar age to me that prioritise factors such as these when seeking places to rent or buy.

I found that actually the final results aren't too suprising!

#### Data Selection

First of all, I will be using recent data (2018/19) from the London Datastore to rank each of the 33 London boroughs based on the reported happiness of people who live there, and the percentage of the borough that is "green" (parks, trees). I chose this data because it is readily available in a consistent .csv format, and because it is relatively recent. 

For the last part, number of coffee shops, I will be making use of the Foursquare API to search for coffee shops within 1km of the centre of the borough. I will use a geocoder to define a longitude and latitude for each borough, before passing this to the API in a search. 


#### PART ONE - LOADING SURVEY DATA

In [1]:
# Firstly, load libraries that may be required

import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 

!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim

import requests
import random
from pandas.io.json import json_normalize 
from IPython.display import Image 
from IPython.core.display import HTML 
import matplotlib.cm as cm 
import matplotlib.colors as colors
from sklearn.cluster import KMeans 

!conda install -c conda-forge folium=0.5.0 --yes 
import folium 

print('Libraries imported.')

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.

Libraries imported.


In [2]:
# Import data, Happiness by borough

dfH = pd.read_csv (r'personalWellbeingBoroughMeanHappiness.csv')

# Clean up the Dataframe by removing gaps, rename columns...

dfH = dfH.drop([0,1, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51], axis=0)
dfH = dfH.drop(["Happiness", "Unnamed: 3", "Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8"], axis=1)
dfH.columns = ['BoroughCode', 'Borough', 'HappinessMeanScore']

# Adding a ranking based on PercentGreenSpace and sort...

dfH['BoroughRankHappiness']=dfH['HappinessMeanScore'].rank(ascending=0)
dfH = dfH.sort_values(by=['BoroughRankHappiness'], ascending=True)
dfH.head()


,BoroughCode,Borough,HappinessMeanScore,BoroughRankHappiness
6,E09000005,Brent,7.84,1.0
30,E09000029,Sutton,7.79,2.0
9,E09000008,Croydon,7.78,3.0
19,E09000018,Hounslow,7.73,4.5
31,E09000030,Tower Hamlets,7.73,4.5


In [3]:
# Import data, Green spaces by borough 

dfG = pd.read_csv (r'londonBoroughStatsGreen.csv')

# Clean up the Dataframe by removing gaps, rename columns...

dfG = dfG.drop(["borough_area_hectare", "green_area_hectare", "blue _area_hectare", "green+blue_area_hectare", "percent_blue", "percent_green+blue"], axis=1)
dfG.columns = ['BoroughCode', 'Borough', 'PercentGreenSpace']

# Adding a ranking based on PercentGreenSpace and sort...

dfG['BoroughRankGreen']=dfG['PercentGreenSpace'].rank(ascending=0)
dfG = dfG.sort_values(by=['BoroughRankGreen'], ascending=True)
dfG.head()


,BoroughCode,Borough,PercentGreenSpace,BoroughRankGreen
5,E09000006,Bromley,68.36,1.0
15,E09000016,Havering,65.98,2.0
26,E09000027,Richmond upon Thames,61.99,3.0
2,E09000003,Barnet,57.99,4.0
7,E09000008,Croydon,55.53,5.0


In [106]:
# Now I need to merge the dataframes in to one, and add "London" to the boroughs to ensure the geocoder works in the next bit.

dfM = pd.merge(left=dfG, right=dfH, how='left', left_on='Borough', right_on='Borough')
dfM = dfM.drop(["BoroughCode_x", "BoroughCode_y"], axis=1)
dfM['Borough'] = (dfM['Borough'] + ', London')
dfM.head()

# Nice! So far so good.

,Borough,PercentGreenSpace,BoroughRankGreen,HappinessMeanScore,BoroughRankHappiness
0,"Bromley, London",68.36,1.0,7.47,19.0
1,"Havering, London",65.98,2.0,7.4,22.0
2,"Richmond upon Thames, London",61.99,3.0,7.48,18.0
3,"Barnet, London",57.99,4.0,7.44,21.0
4,"Croydon, London",55.53,5.0,7.78,3.0


#### PART TWO - FOURSQUARE DATA

Now that I have a ranking for each Borough based on recent survey data, I will get data from Foursquare. I want to find out how many coffee shops there are in each borough.

#### Pre-processing - giving each borough a latitude and longitude

In [22]:
from geopy.extra.rate_limiter import RateLimiter
locator = Nominatim(user_agent="foursquare_agent")

# delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)

# add location column

dfM['Geolocation'] = dfM['Borough'].apply(geocode)

# create longitude and latitude, split into separate columns

dfM['point'] = dfM['Geolocation'].apply(lambda loc: tuple(loc.point) if loc else None)

dfM[['Latitude', 'Longitude', 'Altitude']] = pd.DataFrame(dfM['point'].tolist(), index=dfM.index)
dfM = dfM.drop(["Altitude"], axis=1)
dfM.head()

# Nice! These all look to be London Latitudes!


,Borough,PercentGreenSpace,BoroughRankGreen,HappinessMeanScore,BoroughRankHappiness,Geolocation,point,Latitude,Longitude
0,"Bromley, London",68.36,1.0,7.47,19.0,"(Bromley, London, Greater London, England, BR1...","(51.4028046, 0.0148142, 0.0)",51.402805,0.014814
1,"Havering, London",65.98,2.0,7.4,22.0,"(Havering, Lewis Street, Maitland Park, Kentis...","(51.5443687, -0.1443031798919795, 0.0)",51.544369,-0.144303
2,"Richmond upon Thames, London",61.99,3.0,7.48,18.0,"(London Borough of Richmond upon Thames, Londo...","(51.44037225, -0.3057195504470487, 0.0)",51.440372,-0.305720
3,"Barnet, London",57.99,4.0,7.44,21.0,"(Chipping Barnet, London Borough of Barnet, Lo...","(51.65309, -0.2002261, 0.0)",51.653090,-0.200226
4,"Croydon, London",55.53,5.0,7.78,3.0,"(Croydon, London, Greater London, England, CR0...","(51.3713049, -0.101957, 0.0)",51.371305,-0.101957


In [20]:
# Nice! These all look to be London latitudes. Let's put it on a map to be sure there isn't anything erroneous... 

dfMmap = folium.Map(
    location=[51.5074,0.1278],
    tiles='cartodbpositron',
    zoom_start=9,
)
dfM.apply(lambda row:folium.CircleMarker(location=[row["Latitude"], row["Longitude"]]).add_to(dfMmap), axis=1)
dfMmap

#### Now that the data is all in one table, and we have geolocations, we can make use of the Foursquare API.

In [49]:
# Define Foursquare credentials and version

CLIENT_ID = 'AKMQQEERJSWPDELH31A0BY1HFQSOEJETRLBS2OIF52NKMIVU' # your Foursquare ID
CLIENT_SECRET = 'V4SSZ2KBLU5Y00HNVBD3UKYKKEF104BJHQI2YGPHKCR0MDLQ' # your Foursquare Secret
VERSION = '20180604'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

search_query = 'Coffee'
radius = 1000
latitude = 51.402805
longitude = 0.014814

Your credentails:
CLIENT_ID: AKMQQEERJSWPDELH31A0BY1HFQSOEJETRLBS2OIF52NKMIVU
CLIENT_SECRET:V4SSZ2KBLU5Y00HNVBD3UKYKKEF104BJHQI2YGPHKCR0MDLQ


'https://api.foursquare.com/v2/venues/search?client_id=AKMQQEERJSWPDELH31A0BY1HFQSOEJETRLBS2OIF52NKMIVU&client_secret=V4SSZ2KBLU5Y00HNVBD3UKYKKEF104BJHQI2YGPHKCR0MDLQ&ll=51.402805,0.014814&v=20180604&query=Coffee&radius=1000&limit=30'

In [90]:
# urls required to search .... please kindly ignore the fact that this is huge! I will update once I am able
# to get a loop to work for this purpose!! 

url1 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[0], dfM['Longitude'].iloc[0], VERSION, search_query, radius, LIMIT))
results1 = requests.get(url1).json()
venues1 = results1['response']['venues']
coffeeResults1 = pd.json_normalize(venues1)
coffeeResultsCount1 = coffeeResults1.shape[0]

url2 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[1], dfM['Longitude'].iloc[1], VERSION, search_query, radius, LIMIT))
results2 = requests.get(url2).json()
venues2 = results2['response']['venues']
coffeeResults2 = pd.json_normalize(venues2)
coffeeResultsCount2 = coffeeResults2.shape[0]

url3 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[2], dfM['Longitude'].iloc[2], VERSION, search_query, radius, LIMIT))
results3 = requests.get(url3).json()
venues3 = results3['response']['venues']
coffeeResults3 = pd.json_normalize(venues3)
coffeeResultsCount3 = coffeeResults3.shape[0]

url4 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[3], dfM['Longitude'].iloc[3], VERSION, search_query, radius, LIMIT))
results4 = requests.get(url4).json()
venues4 = results4['response']['venues']
coffeeResults4 = pd.json_normalize(venues4)
coffeeResultsCount4 = coffeeResults4.shape[0]

url5 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[4], dfM['Longitude'].iloc[4], VERSION, search_query, radius, LIMIT))
results5 = requests.get(url5).json()
venues5 = results5['response']['venues']
coffeeResults5 = pd.json_normalize(venues5)
coffeeResultsCount5 = coffeeResults5.shape[0]

url6 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[5], dfM['Longitude'].iloc[5], VERSION, search_query, radius, LIMIT))
results6 = requests.get(url6).json()
venues6 = results6['response']['venues']
coffeeResults6 = pd.json_normalize(venues6)
coffeeResultsCount6 = coffeeResults6.shape[0]

url7 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[6], dfM['Longitude'].iloc[6], VERSION, search_query, radius, LIMIT))
results7 = requests.get(url7).json()
venues7 = results7['response']['venues']
coffeeResults7 = pd.json_normalize(venues7)
coffeeResultsCount7 = coffeeResults7.shape[0]

url8 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[7], dfM['Longitude'].iloc[7], VERSION, search_query, radius, LIMIT))
results8 = requests.get(url8).json()
venues8 = results8['response']['venues']
coffeeResults8 = pd.json_normalize(venues8)
coffeeResultsCount8 = coffeeResults8.shape[0]

url9 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[8], dfM['Longitude'].iloc[8], VERSION, search_query, radius, LIMIT))
results9 = requests.get(url9).json()
venues9 = results9['response']['venues']
coffeeResults9 = pd.json_normalize(venues9)
coffeeResultsCount9 = coffeeResults9.shape[0]

url10 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[9], dfM['Longitude'].iloc[9], VERSION, search_query, radius, LIMIT))
results10 = requests.get(url10).json()
venues10 = results10['response']['venues']
coffeeResults10 = pd.json_normalize(venues10)
coffeeResultsCount10 = coffeeResults10.shape[0]

url11 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[10], dfM['Longitude'].iloc[10], VERSION, search_query, radius, LIMIT))
results11 = requests.get(url11).json()
venues11 = results11['response']['venues']
coffeeResults11 = pd.json_normalize(venues11)
coffeeResultsCount11 = coffeeResults11.shape[0]

url12 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[11], dfM['Longitude'].iloc[11], VERSION, search_query, radius, LIMIT))
results12 = requests.get(url12).json()
venues12 = results12['response']['venues']
coffeeResults12 = pd.json_normalize(venues12)
coffeeResultsCount12 = coffeeResults12.shape[0]

url13 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[12], dfM['Longitude'].iloc[12], VERSION, search_query, radius, LIMIT))
results13 = requests.get(url13).json()
venues13 = results13['response']['venues']
coffeeResults13 = pd.json_normalize(venues13)
coffeeResultsCount13 = coffeeResults13.shape[0]

url14 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[13], dfM['Longitude'].iloc[13], VERSION, search_query, radius, LIMIT))
results14 = requests.get(url14).json()
venues14 = results14['response']['venues']
coffeeResults14 = pd.json_normalize(venues14)
coffeeResultsCount14 = coffeeResults14.shape[0]

url15 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[14], dfM['Longitude'].iloc[14], VERSION, search_query, radius, LIMIT))
results15 = requests.get(url15).json()
venues15 = results15['response']['venues']
coffeeResults15 = pd.json_normalize(venues15)
coffeeResultsCount15 = coffeeResults15.shape[0]

url16 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[15], dfM['Longitude'].iloc[15], VERSION, search_query, radius, LIMIT))
results16 = requests.get(url16).json()
venues16 = results16['response']['venues']
coffeeResults16 = pd.json_normalize(venues16)
coffeeResultsCount16 = coffeeResults16.shape[0]

url17 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[16], dfM['Longitude'].iloc[16], VERSION, search_query, radius, LIMIT))
results17 = requests.get(url17).json()
venues17 = results17['response']['venues']
coffeeResults17 = pd.json_normalize(venues17)
coffeeResultsCount17 = coffeeResults17.shape[0]

url18 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[17], dfM['Longitude'].iloc[17], VERSION, search_query, radius, LIMIT))
results18 = requests.get(url18).json()
venues18 = results18['response']['venues']
coffeeResults18 = pd.json_normalize(venues18)
coffeeResultsCount18 = coffeeResults18.shape[0]

url19 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[18], dfM['Longitude'].iloc[18], VERSION, search_query, radius, LIMIT))
results19 = requests.get(url19).json()
venues19 = results19['response']['venues']
coffeeResults19 = pd.json_normalize(venues19)
coffeeResultsCount19 = coffeeResults19.shape[0]

url20 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[19], dfM['Longitude'].iloc[19], VERSION, search_query, radius, LIMIT))
results20 = requests.get(url20).json()
venues20 = results20['response']['venues']
coffeeResults20 = pd.json_normalize(venues20)
coffeeResultsCount20 = coffeeResults20.shape[0]

url21 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[20], dfM['Longitude'].iloc[20], VERSION, search_query, radius, LIMIT))
results21 = requests.get(url21).json()
venues21 = results21['response']['venues']
coffeeResults21 = pd.json_normalize(venues21)
coffeeResultsCount21 = coffeeResults21.shape[0]

url22 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[21], dfM['Longitude'].iloc[21], VERSION, search_query, radius, LIMIT))
results22 = requests.get(url22).json()
venues22 = results22['response']['venues']
coffeeResults22 = pd.json_normalize(venues22)
coffeeResultsCount22 = coffeeResults22.shape[0]

url23 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[22], dfM['Longitude'].iloc[22], VERSION, search_query, radius, LIMIT))
results23 = requests.get(url23).json()
venues23 = results23['response']['venues']
coffeeResults23 = pd.json_normalize(venues23)
coffeeResultsCount23 = coffeeResults23.shape[0]

url24 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[23], dfM['Longitude'].iloc[23], VERSION, search_query, radius, LIMIT))
results24 = requests.get(url24).json()
venues24 = results24['response']['venues']
coffeeResults24 = pd.json_normalize(venues24)
coffeeResultsCount24 = coffeeResults24.shape[0]

url25 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[24], dfM['Longitude'].iloc[24], VERSION, search_query, radius, LIMIT))
results25 = requests.get(url25).json()
venues25 = results25['response']['venues']
coffeeResults25 = pd.json_normalize(venues25)
coffeeResultsCount25 = coffeeResults25.shape[0]

url26 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[25], dfM['Longitude'].iloc[25], VERSION, search_query, radius, LIMIT))
results26 = requests.get(url26).json()
venues26 = results26['response']['venues']
coffeeResults26 = pd.json_normalize(venues26)
coffeeResultsCount26 = coffeeResults26.shape[0]

url27 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[26], dfM['Longitude'].iloc[26], VERSION, search_query, radius, LIMIT))
results27 = requests.get(url27).json()
venues27 = results27['response']['venues']
coffeeResults27 = pd.json_normalize(venues27)
coffeeResultsCount27 = coffeeResults27.shape[0]

url28 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[27], dfM['Longitude'].iloc[27], VERSION, search_query, radius, LIMIT))
results28 = requests.get(url28).json()
venues28 = results28['response']['venues']
coffeeResults28 = pd.json_normalize(venues28)
coffeeResultsCount28 = coffeeResults28.shape[0]

url29 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[28], dfM['Longitude'].iloc[28], VERSION, search_query, radius, LIMIT))
results29 = requests.get(url29).json()
venues29 = results29['response']['venues']
coffeeResults29 = pd.json_normalize(venues29)
coffeeResultsCount29 = coffeeResults29.shape[0]

url30 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[29], dfM['Longitude'].iloc[29], VERSION, search_query, radius, LIMIT))
results30 = requests.get(url30).json()
venues30 = results30['response']['venues']
coffeeResults30 = pd.json_normalize(venues30)
coffeeResultsCount30 = coffeeResults30.shape[0]

url31 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[30], dfM['Longitude'].iloc[30], VERSION, search_query, radius, LIMIT))
results31 = requests.get(url31).json()
venues31 = results31['response']['venues']
coffeeResults31 = pd.json_normalize(venues31)
coffeeResultsCount31 = coffeeResults31.shape[0]

url32 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[31], dfM['Longitude'].iloc[31], VERSION, search_query, radius, LIMIT))
results32 = requests.get(url32).json()
venues32 = results32['response']['venues']
coffeeResults32 = pd.json_normalize(venues32)
coffeeResultsCount32 = coffeeResults32.shape[0]

url33 = ('https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, dfM['Latitude'].iloc[32], dfM['Longitude'].iloc[32], VERSION, search_query, radius, LIMIT))
results33 = requests.get(url33).json()
venues33 = results33['response']['venues']
coffeeResults33 = pd.json_normalize(venues33)
coffeeResultsCount33 = coffeeResults33.shape[0]

# phew... finally.

In [107]:
# make a new dataframe with the borough and the coffeeresultscount, then add to the master dataframe.

boroughCoffee = {'Borough': [[dfM['Borough'].iloc[0]], [dfM['Borough'].iloc[1]], [dfM['Borough'].iloc[2]], [dfM['Borough'].iloc[3]], [dfM['Borough'].iloc[4]], [dfM['Borough'].iloc[5]], [dfM['Borough'].iloc[6]], [dfM['Borough'].iloc[7]], [dfM['Borough'].iloc[8]], [dfM['Borough'].iloc[9]], [dfM['Borough'].iloc[10]], [dfM['Borough'].iloc[11]], [dfM['Borough'].iloc[12]], [dfM['Borough'].iloc[13]], [dfM['Borough'].iloc[14]], [dfM['Borough'].iloc[15]], [dfM['Borough'].iloc[16]], [dfM['Borough'].iloc[17]], [dfM['Borough'].iloc[18]], [dfM['Borough'].iloc[19]], [dfM['Borough'].iloc[20]], [dfM['Borough'].iloc[21]], [dfM['Borough'].iloc[22]], [dfM['Borough'].iloc[23]], [dfM['Borough'].iloc[24]], [dfM['Borough'].iloc[25]], [dfM['Borough'].iloc[26]], [dfM['Borough'].iloc[27]], [dfM['Borough'].iloc[28]], [dfM['Borough'].iloc[29]], [dfM['Borough'].iloc[30]], [dfM['Borough'].iloc[31]], [dfM['Borough'].iloc[32]]],
                'CoffeeShops': [coffeeResultsCount1, coffeeResultsCount2, coffeeResultsCount3, coffeeResultsCount4, coffeeResultsCount5, coffeeResultsCount6, coffeeResultsCount7, coffeeResultsCount8, coffeeResultsCount9, coffeeResultsCount10, coffeeResultsCount11, coffeeResultsCount12, coffeeResultsCount13, coffeeResultsCount14, coffeeResultsCount15, coffeeResultsCount16, coffeeResultsCount17, coffeeResultsCount18, coffeeResultsCount19, coffeeResultsCount20, coffeeResultsCount21, coffeeResultsCount22, coffeeResultsCount23, coffeeResultsCount24, coffeeResultsCount25, coffeeResultsCount26, coffeeResultsCount27, coffeeResultsCount28, coffeeResultsCount29, coffeeResultsCount30, coffeeResultsCount31, coffeeResultsCount32, coffeeResultsCount33]
               }

dfBC = pd.DataFrame(boroughCoffee, columns = ['Borough', 'CoffeeShops'])
dfBC['Borough'] = dfBC['Borough'].map(lambda x: str(x)[:-2])
dfBC['Borough'] = dfBC['Borough'].map(lambda x: str(x)[2:])
dfBC['CoffeeRank']=dfBC['CoffeeShops'].rank(ascending=0)

dfM = pd.merge(left=dfM, right=dfBC, how='left', left_on='Borough', right_on='Borough')
dfM.head()

,Borough,PercentGreenSpace,BoroughRankGreen,HappinessMeanScore,BoroughRankHappiness,CoffeeShops,CoffeeRank
0,"Bromley, London",68.36,1.0,7.47,19.0,6,22.0
1,"Havering, London",65.98,2.0,7.4,22.0,30,4.0
2,"Richmond upon Thames, London",61.99,3.0,7.48,18.0,2,28.0
3,"Barnet, London",57.99,4.0,7.44,21.0,4,26.5
4,"Croydon, London",55.53,5.0,7.78,3.0,19,11.0


#### Now I have a dataframe that has all 33 boroughs, each with a ranking based on green space, reported happiness, and number of coffee shops. I will add up the ranks to give a final rank, with the smallest rank being the most "pleasant" neighbourhood!

In [115]:
dfM['OverallScore']=dfM['BoroughRankGreen']+dfM['BoroughRankHappiness']+dfM['CoffeeRank']
dfM

dfFINAL = dfM.drop(["PercentGreenSpace", "HappinessMeanScore", "CoffeeShops"], axis=1)
dfFINAL['FINAL_RANK']=dfFINAL['OverallScore'].rank(ascending=1)
dfFINAL = dfFINAL.sort_values(by=['FINAL_RANK'], ascending=True)
dfFINAL

,Borough,BoroughRankGreen,BoroughRankHappiness,CoffeeRank,OverallScore,FINAL_RANK
4,"Croydon, London",5.0,3.0,11.0,19.0,1.0
1,"Havering, London",2.0,22.0,4.0,28.0,2.0
9,"Kingston upon Thames, London",10.0,11.0,11.0,32.0,3.0
14,"Ealing, London",15.0,8.5,13.5,37.0,4.0
12,"Hounslow, London",13.0,4.5,22.0,39.5,5.5
11,"Merton, London",12.0,12.0,15.5,39.5,5.5
20,"Brent, London",21.0,1.0,19.0,41.0,7.5
5,"Harrow, London",6.0,8.5,26.5,41.0,7.5
0,"Bromley, London",1.0,19.0,22.0,42.0,9.0
10,"Sutton, London",11.0,2.0,30.0,43.0,10.0


In [116]:
dfFINAL.head()

,Borough,BoroughRankGreen,BoroughRankHappiness,CoffeeRank,OverallScore,FINAL_RANK
4,"Croydon, London",5.0,3.0,11.0,19.0,1.0
1,"Havering, London",2.0,22.0,4.0,28.0,2.0
9,"Kingston upon Thames, London",10.0,11.0,11.0,32.0,3.0
14,"Ealing, London",15.0,8.5,13.5,37.0,4.0
12,"Hounslow, London",13.0,4.5,22.0,39.5,5.5


#### And there we have it! The Pleasantness Index indicates that Croydon is the most pleasant borough to live in, followed by Havering, Kingston upon Thames, Ealing and then Hounslow. It would be interesting to think of more "pleasant" factors and build this index out further in the future. 

#### Thanks for reading! 